<a href="https://colab.research.google.com/github/pravallika16-in/firstRepo/blob/master/final.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Conv2D,MaxPooling2D
from keras.layers.normalization import BatchNormalization
from keras.layers import Activation, Dropout, Flatten, Dense
from keras import backend as K
import numpy as np
from keras.preprocessing import image
from keras.optimizers import Adam

In [ ]:
img_width, img_height = 256,256

train_data_dir = '/content/drive/MyDrive/imgs/train'
test_data_dir = '/content/drive/MyDrive/imgs/validation'
no_train_samples = 17943
no_test_samples = 4481
epochs =15
batch_size=64

In [ ]:
if K.image_data_format() =='channels_first':
    input_shape= (3,img_width,img_height)
else:
    input_shape =(img_width, img_height,3)

In [ ]:
train_datagen = ImageDataGenerator(
    rescale=1. /255,
    shear_range = 0.2,
    zoom_range =0.2,
    horizontal_flip=True,
    rotation_range=25,
    width_shift_range=0.1,
    height_shift_range=0.1,
    fill_mode="nearest"
    )

In [ ]:
test_datagen = ImageDataGenerator(rescale=1. /255)

In [ ]:
train_generator = train_datagen.flow_from_directory(
     train_data_dir,
    target_size=(img_width,img_height),
    batch_size= batch_size,
    class_mode='categorical'
)

Found 17943 images belonging to 10 classes.


In [ ]:
validation_generator = test_datagen.flow_from_directory(
     test_data_dir,
    target_size=(img_width,img_height),
    batch_size= batch_size,
    class_mode='categorical'
)

Found 4481 images belonging to 10 classes.


In [ ]:
model=Sequential()

model.add(Conv2D(64,(3,3), padding='same', input_shape= input_shape))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.25))

model.add(Conv2D(128,(5,5), padding='same'))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.25))

model.add(Conv2D(512,(3,3), padding='same'))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.25))

model.add(Conv2D(128,(3,3), padding='same'))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.25))

model.add(Flatten())
model.add(Dense(128))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(Dropout(0.25))
model.add(Dense(1000))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(Dropout(0.25))

model.add(Dense(10, activation='softmax'))
opt=Adam(lr=0.0005)
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 256, 256, 64)      1792      
_________________________________________________________________
batch_normalization (BatchNo (None, 256, 256, 64)      256       
_________________________________________________________________
activation (Activation)      (None, 256, 256, 64)      0         
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 128, 128, 64)      0         
_________________________________________________________________
dropout (Dropout)            (None, 128, 128, 64)      0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 128, 128, 128)     204928    
_________________________________________________________________
batch_normalization_1 (Batch (None, 128, 128, 128)     5

/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/optimizer_v2/optimizer_v2.py:375: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  "The `lr` argument is deprecated, use `learning_rate` instead.")


In [ ]:
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

model.summary()

In [ ]:

from keras.callbacks import ModelCheckpoint,ReduceLROnPlateau


checkpoint=ModelCheckpoint("model_weights.h5", monitor="val_accuracy", save_weights_only=True, model='max', verbose=1)
reduce_lr = ReduceLROnPlateau(monitor = 'val_loss', factor=0.1, patience=2, min_lr= 0.00001, model= 'auto')

In [ ]:
model.fit(
    train_generator,
    steps_per_epoch=no_train_samples // batch_size,
    epochs = epochs,
    validation_data = validation_generator,
    validation_steps= no_test_samples // batch_size
)

Epoch 1/15
280/280 [==============================] - 5519s 20s/step - loss: 2.3012 - accuracy: 0.2271 - val_loss: 4.0212 - val_accuracy: 0.1185
Epoch 2/15
280/280 [==============================] - 294s 1s/step - loss: 1.3386 - accuracy: 0.5324 - val_loss: 1.1314 - val_accuracy: 0.6547
Epoch 3/15
280/280 [==============================] - 294s 1s/step - loss: 0.9242 - accuracy: 0.6805 - val_loss: 6.8289 - val_accuracy: 0.2277
Epoch 4/15
280/280 [==============================] - 294s 1s/step - loss: 0.7406 - accuracy: 0.7447 - val_loss: 6.0391 - val_accuracy: 0.2884
Epoch 5/15
280/280 [==============================] - 295s 1s/step - loss: 0.6325 - accuracy: 0.7844 - val_loss: 5.0886 - val_accuracy: 0.3313
Epoch 6/15
280/280 [==============================] - 294s 1s/step - loss: 0.5384 - accuracy: 0.8152 - val_loss: 2.3119 - val_accuracy: 0.5342
Epoch 7/15
280/280 [==============================] - 294s 1s/step - loss: 0.4830 - accuracy: 0.8358 - val_loss: 4.7076 - val_accuracy: 0.33

In [ ]:
model.save('/content/drive/MyDrive/third.model')

INFO:tensorflow:Assets written to: /content/drive/MyDrive/third.model/assets


In [ ]:
from tensorflow.keras.models import load_model
model=load_model('/content/drive/MyDrive/third.model')